<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Part 1:  (Web) Scrape Wikipedia Page on Toronto Neighbourhoods


#### Install Python Packages

In [1]:
#install Beatiful Soup (if not yet available)

!pip install beautifulsoup4

     |████████████████████████████████| 102kB 17.6MB/s ta 0:00:01


In [2]:
#install xml parser(if not yet available)

!pip install lxml

     |████████████████████████████████| 5.8MB 20.2MB/s eta 0:00:01


In [3]:
#install HTML5 library (if not yet available)

!pip install html5lib

#### Import Python Libraries

In [4]:
#import libraries


import pandas as pd
import requests
from bs4 import BeautifulSoup

import html

In [5]:
#get html and 'soupify' table

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')#'html5lib')
#print(soup.prettify)

In [6]:
#access table

table = soup.find('table')
data = pd.read_html(str(table))

In [7]:
#check html to retrieved
data

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns]]

In [8]:
#place table contents into datafrme
df = data[0]

In [9]:
#verify data type

print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [10]:
#display dataframe

df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [11]:
#copy original dataframe in oder to process it
dfp = df

#dfp



#### Build the DataFrame

In [12]:
#set up the dataframe columns for appending

dfp_p = dfp['Postcode']
dfp_b = dfp['Borough']
dfp_n = dfp['Neighbourhood']

In [13]:
#convert series to dataframe

dfp_n = dfp_n.to_frame()
dfp_b = dfp_b.to_frame()
dfp_p = dfp_p.to_frame()

In [14]:
#combine 'slices' into a single dataframe

dfp_bn = dfp_b.merge(dfp_n,left_index=True, right_index=True)
dfp_pbn = dfp_p.merge(dfp_bn,left_index=True, right_index=True)

#### Handle Boroughs that are "Not assigned" 

In [15]:
#remove all Boroughs that are not assigned
dfp_pbn = dfp_pbn[dfp_pbn['Borough'] != "Not assigned"]

print(len(dfp_pbn))

211


#### Handle Neighbourhoods that are "Not assigned" 

In [16]:
#assign Borough value to Neigborhoods that are "Not Assigned"

#initilise variables used in the loop

i=0
nn=0

dfp_pbn_bna = dfp_pbn
rows = len(dfp_pbn_bna)

na = "Not assigned"

#loop through entire dataframe
while i < rows:
    b = dfp_pbn_bna['Borough'].values[i]
    n = dfp_pbn_bna['Neighbourhood'].values[i]
    n.strip()
    b.strip()
    if (n==na):
        nn ++ 1
        print("N: No borough!",i,b,n)
        dfp_pbn_bna['Neighbourhood'].values[i] = b
    i += 1    
        
print(nn,len(dfp_pbn_bna))

N: No borough! 6 Queen's Park Not assigned
0 211


In [17]:
print(dfp_pbn_bna.values[0])

['M3A' 'North York' 'Parkwoods']


In [18]:
dfp_pbn_bna

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


#### Handle Neiborhoods that belong to the same Postcode

In [19]:
#join Neighborhoods with the same Postcode

#setup the variables used in the loop
i=0
i2=0
np=0
dfp_pbn_bna_nna = dfp_pbn_bna
rows = len(dfp_pbn_bna_nna)


dfp_pbn_bna_nna.reset_index(drop=True, inplace=True)
#dfp_pbn_bna_nna.head()

dfp_pbn_bna_nna2 = dfp_pbn_bna_nna

#perform a nested loop to concatenate neighborhoods with similar postcodes
while i < rows:
    b = dfp_pbn_bna_nna['Borough'].values[i]
    n = dfp_pbn_bna_nna['Neighbourhood'].values[i]
    p = dfp_pbn_bna_nna['Postcode'].values[i]
    i2 = i + 1
    
    while i2 < rows:
        p2 = dfp_pbn_bna_nna['Postcode'].values[i2]
        if (p == p2): 
            n2 = dfp_pbn_bna_nna['Neighbourhood'].values[i2]
            dfp_pbn_bna_nna['Neighbourhood'].values[i] = n + ", "  +  n2
           
        i2 += 1
    i += 1    
        
print(np,len(dfp_pbn_bna_nna))

0 211


#### Finalise dataframe

In [20]:
dfp_pbn_bna_nna2

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
9,M1B,Scarborough,Malvern


In [21]:
#remove duplicate rowa
dfp_pbn_bna_nna2.drop_duplicates(subset ="Postcode", keep = 'first', inplace = True) 

In [22]:
print(len(dfp_pbn_bna_nna2))

103


In [23]:
#reset dtaframe to start rows at 0
dfp_pbn_bna_nna2.reset_index(drop=True, inplace=True)

In [24]:
#rename coumn names
dfp_pbn_bna_nna2.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"},inplace=True)
dfp_pbn_bna_nna2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Final step (Part 1 of 3)

In [25]:
#display number of rows and columns
dfp_pbn_bna_nna2.shape

(103, 3)

## Part 2: Geospatial daea containing latitudes & longitudes

#### First get the csv to make it work

In [26]:
#download csv 

!wget -q -O 'TorHoods_Coord.csv' : http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [27]:
#put csv into dataframe
df_LatLong_csv = pd.read_csv('TorHoods_Coord.csv')
df_LatLong_csv.rename(columns={"Postal Code": "PostalCode"},inplace=True)

#df_LatLong_csv

#### Make final dataframe

In [28]:
#get Boroughs
df_b = dfp_pbn_bna_nna2[['PostalCode','Borough']]
df_b


,PostalCode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
3,M6A,North York
4,M7A,Queen's Park
5,M9A,Etobicoke
6,M1B,Scarborough
7,M3B,North York
8,M4B,East York
9,M5B,Downtown Toronto


#### Boroughs

In [29]:
#add borough rows to original dataframe
df_m1 = df_LatLong_csv
df_m1 = df_m1.merge(df_b, how='inner',on ='PostalCode',left_index=True, right_index=True)
df_m1

,PostalCode,Latitude,Longitude,Borough
0,M1B,43.806686,-79.194353,North York
1,M1C,43.784535,-79.160497,North York
2,M1E,43.763573,-79.188711,Downtown Toronto
3,M1G,43.770992,-79.216917,North York
4,M1H,43.773136,-79.239476,Queen's Park
5,M1J,43.744734,-79.239476,Etobicoke
6,M1K,43.727929,-79.262029,Scarborough
7,M1L,43.711112,-79.284577,North York
8,M1M,43.716316,-79.239476,East York
9,M1N,43.692657,-79.264848,Downtown Toronto


#### Neighborhoods

In [30]:
#get Neighborhoods
df_n = dfp_pbn_bna_nna2[['PostalCode','Neighborhood']]
df_n


,PostalCode,Neighborhood
0,M3A,Parkwoods
1,M4A,Victoria Village
2,M5A,"Harbourfront, Regent Park"
3,M6A,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park
5,M9A,Islington Avenue
6,M1B,"Rouge, Malvern"
7,M3B,Don Mills North
8,M4B,"Woodbine Gardens, Parkview Hill"
9,M5B,"Ryerson, Garden District"


#### Combine into one

In [31]:
#add neighbourhood rows to intermediary dataframe
df_m2 = df_m1
df_m2 = df_m2.merge(df_n, how='inner',on ='PostalCode',left_index=True, right_index=True)
df_m2

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,North York,Parkwoods
1,M1C,43.784535,-79.160497,North York,Victoria Village
2,M1E,43.763573,-79.188711,Downtown Toronto,"Harbourfront, Regent Park"
3,M1G,43.770992,-79.216917,North York,"Lawrence Heights, Lawrence Manor"
4,M1H,43.773136,-79.239476,Queen's Park,Queen's Park
5,M1J,43.744734,-79.239476,Etobicoke,Islington Avenue
6,M1K,43.727929,-79.262029,Scarborough,"Rouge, Malvern"
7,M1L,43.711112,-79.284577,North York,Don Mills North
8,M1M,43.716316,-79.239476,East York,"Woodbine Gardens, Parkview Hill"
9,M1N,43.692657,-79.264848,Downtown Toronto,"Ryerson, Garden District"


#### Column order

In [32]:
#reorder columns of the dataframe 
df_m2[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,North York,Parkwoods,43.806686,-79.194353
1,M1C,North York,Victoria Village,43.784535,-79.160497
2,M1E,Downtown Toronto,"Harbourfront, Regent Park",43.763573,-79.188711
3,M1G,North York,"Lawrence Heights, Lawrence Manor",43.770992,-79.216917
4,M1H,Queen's Park,Queen's Park,43.773136,-79.239476
5,M1J,Etobicoke,Islington Avenue,43.744734,-79.239476
6,M1K,Scarborough,"Rouge, Malvern",43.727929,-79.262029
7,M1L,North York,Don Mills North,43.711112,-79.284577
8,M1M,East York,"Woodbine Gardens, Parkview Hill",43.716316,-79.239476
9,M1N,Downtown Toronto,"Ryerson, Garden District",43.692657,-79.264848


#### Try using geocoder instead of csv

#### Make sure package is installed

In [33]:
!pip install geocoder

     |████████████████████████████████| 102kB 17.6MB/s ta 0:00:01
     |████████████████████████████████| 829kB 29.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 4.7MB/s eta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


#### Import library

In [34]:
# import geocoder
import geocoder 

#### Get one to test

In [35]:
bd = dfp_pbn_bna_nna2['Borough'].values[0]
print(bd)

North York


#### Use string format prescribed by the assement - some code on the Internet doesn't have the curly braces at the start

In [36]:
bstr = '{}, ' + bd  + ', Ontario''.format(postal_code))'
print(bstr)

{}, North York, Ontario.format(postal_code))


In [37]:
# initialize your variable to None
lat_lng_coords = None

In [38]:
# loop until you get the coordinates
# Comment out because it keeps the kernel busy
#while(lat_lng_coords is None):
#  g = geocoder.google(bstr)
#  lat_lng_coords = g.latlng

In [39]:
# to avoid an error this was commented out because the code above was not executed
# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

#### it's taking sometime to process just one so I'm sticking with the csv - end of Part 2

## Part 3:  Analysis of Neighbohoods

#### Install packages

In [40]:
#install geopy if not yet installed
!pip install geopy

     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


#### Import libraries

In [41]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Import Nominatim

In [42]:
from geopy.geocoders import Nominatim

#### Construct dataframe as the source for the map

#### Borough limitation to those with Toronto as part of its name justification

1.) Familiarity.  While I live far from Canda, I know of Toronto.  Moreover, it may facilitate the explanation of the location to other people.

2.) Numbers. 25 Neighborhood out of the culled 103 seems statiscally valid.  Moreover out of the original 288,it still seems significant.

3.) Relative Ease. It is more straightforward to identify the Borough with Toronto - it is also handy that you can leverage an existing Python method for this.

4.) Suggestion.  Aside from the facilitator recommending to do this, I seem to recall reading that this was part of the assessment specifications at one point. 

#### Get dataframe with Latitude and Longitude info 

In [42]:
#asign Lat & Long
df_am = df_LatLong_csv


In [43]:
# check contents of original datafram: one wih neighborhoods seperated
dfp

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [44]:
#assign Lat & Long df for processing

df_am2 = df_am
df_am2

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


####  Make sure Postcode column name is the same before merging

In [45]:

dfp.rename(columns={"Postcode":"PostalCode" }, inplace=True)
dfp.head()#verify that the column name is PostalCode 

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [48]:

df_am2.rename(columns={"Postcode":"PostalCode" }, inplace=True)
df_am2.head()# verify that the column name is PostalCode

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
# combine original dataframe with the one containing latitude & longitude
df_am2 = df_am2.merge(dfp, how='inner',on ='PostalCode',left_index=True, right_index=True)
df_am2.head()#check merge


,PostalCode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Not assigned,Not assigned
1,M1C,43.784535,-79.160497,Not assigned,Not assigned
2,M1E,43.763573,-79.188711,North York,Parkwoods
3,M1G,43.770992,-79.216917,North York,Victoria Village
4,M1H,43.773136,-79.239476,Downtown Toronto,Harbourfront


#### Borough not assigned

In [50]:
#remove all Boroughs that are not assigned
df_am2 = df_am2[df_am2['Borough'] != "Not assigned"]

print(len(df_am2))

78


In [51]:
df_am.head()#verify that the column name is Neigbourhood

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Neighborhoods not assigned

In [52]:
#make sure toassign Borough value to Neigborhoods that are "Not Assigned"

#initilise variables used in the loop

i=0
nn=0


rows = len(df_am2)

na = "Not assigned"

#loop through entire dataframe
while i < rows:
    b = df_am2['Borough'].values[i]
    n = df_am2['Neighbourhood'].values[i]
    n.strip()
    b.strip()
    if (b==na):
        nn += 1
        print("N: No borough!",i,b,n)
        df_am2['Neighbourhood'].values[i] = b
        print("Set n!",i,b,n)
    i += 1    
        
print(nn,len(df_am2))

0 78


#### Boroughs containing Toronto

In [53]:
#determine boroughs that end with Toronto
i = 0
c=0
rows = len(df_am2)
ilist = []

while i < rows:
    if df_am2['Borough'].values[i].endswith('Toronto'):
        #print(df_a['Borough'].values[i],i,ilist)
        ilist.append(i)
        c += 1
    i += 1
print(c)
#print(c) #25 Neighorhoods with Boroughs that start but 0 that end with Toronto
#4 Toronto Boroughs found: Downtown, West, East and Central

25


#### Construct map of Toronto

#### Toronto's Co-ordinates

In [54]:
#get Toronto's co-ordinates
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [56]:
#update condo if necessary
#conda update -n base -c defaults conda

In [57]:
# instal folium package if not present
#conda install -c conda-forge folium

#### Create Daaframe for Toronto Borougs

In [58]:
df_am3 = pd.DataFrame()

In [59]:
#make new dataframe
for i in ilist:
    dfr = df_am2.iloc[i]
    #print(i, dfr)
    df_am3 = df_am3.append(dfr)

In [61]:
#reset dtaframe to start rows at 0
df_am3.reset_index(drop=True, inplace=True)

In [63]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = df_am3

neighborhoods.head()#verify

,Borough,Latitude,Longitude,Neighbourhood,PostalCode
0,Downtown Toronto,43.773136,-79.239476,Harbourfront,M1H
1,Downtown Toronto,43.744734,-79.239476,Regent Park,M1J
2,Downtown Toronto,43.803762,-79.363452,Ryerson,M2H
3,Downtown Toronto,43.778517,-79.346556,Garden District,M2J
4,Downtown Toronto,43.725882,-79.315572,St. James Town,M4A


#### Rename columns

In [67]:
neighborhoods.rename(columns={"PostalCode": "Postcode"})
neighborhoods.head()#verify

,Borough,Latitude,Longitude,Neighbourhood,PostalCode
0,Downtown Toronto,43.773136,-79.239476,Harbourfront,M1H
1,Downtown Toronto,43.744734,-79.239476,Regent Park,M1J
2,Downtown Toronto,43.803762,-79.363452,Ryerson,M2H
3,Downtown Toronto,43.778517,-79.346556,Garden District,M2J
4,Downtown Toronto,43.725882,-79.315572,St. James Town,M4A


In [68]:
df_LatLong_csv.rename(columns={"PostalCode": "Postcode"}, inplace = True)
df_LatLong_csv.head()#verify

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
#neighborhoods = neighborhoods.merge(df_LatLong_csv, how='inner',on ='PostalCode',left_index=True, right_index=True)

In [70]:
neighborhoods.head() #all 5 columns

,Borough,Latitude,Longitude,Neighbourhood,PostalCode
0,Downtown Toronto,43.773136,-79.239476,Harbourfront,M1H
1,Downtown Toronto,43.744734,-79.239476,Regent Park,M1J
2,Downtown Toronto,43.803762,-79.363452,Ryerson,M2H
3,Downtown Toronto,43.778517,-79.346556,Garden District,M2J
4,Downtown Toronto,43.725882,-79.315572,St. James Town,M4A


In [71]:
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Use FourSuare API

#### Build URI

In [75]:
CLIENT_ID = 'ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555' # your Foursquare ID
CLIENT_SECRET = 'LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555
CLIENT_SECRET:LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX


In [76]:
# Build and test data source for the map 
df_am3 = neighborhoods
neighborhood_latitude = df_am3['Latitude'].values[0]
neighborhood_longitude = df_am3['Longitude'].values[0]
print(df_am3['Borough'].values[0], neighborhood_latitude, neighborhood_longitude )

Downtown Toronto 43.773136 -79.23947609999999


In [77]:
LIMIT = 100 # limit of number of venues returned by Foursquare API


radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555&client_secret=LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX&v=20180605&ll=43.773136,-79.23947609999999&radius=500&limit=100'

In [78]:
results = requests.get(url).json()
results#json?

{'meta': {'code': 200, 'requestId': '5d5764c789b06a002c53c8df'},
 'response': {'headerLocation': 'Woburn',
  'headerFullLocation': 'Woburn, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.7776360045, 'lng': -79.2332557734104},
   'sw': {'lat': 43.7686359955, 'lng': -79.24569642658957}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b1711a6f964a520cbc123e3',
       'name': 'Federick Restaurant',
       'location': {'address': '1920 Ellesmere Rd',
        'crossStreet': 'at Bellamy Rd. N',
        'lat': 43.77469659057996,
        'lng': -79.24114242818267,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.77469659057996,
          'lng': -79.24114242818267}],
        'distance': 219,

In [79]:
#
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(7)

,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
2,Thai One On,Thai Restaurant,43.774468,-79.241268
3,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
4,TD Canada Trust,Bank,43.774952,-79.241343
5,B&A Bakery,Bakery,43.774391,-79.243877
6,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.775930,-79.235328


In [81]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


#### Explore area(in the terms of the NY lab)
The following was heavily "influenced" by that ptrevious lab

In [82]:
#function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
# get nearby Toronto venues
toronto_venues = getNearbyVenues(names=df_am3['Neighbourhood'], 
                                   latitudes=df_am3['Latitude'],
                                   longitudes=df_am3['Longitude'] 
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place


In [84]:
 print(toronto_venues.shape) 
toronto_venues.head()

(677, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
1,Harbourfront,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
2,Harbourfront,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
3,Harbourfront,43.773136,-79.239476,Centennial Recreation Centre,43.774593,-79.236500,Athletics & Sports
4,Harbourfront,43.773136,-79.239476,TD Canada Trust,43.774952,-79.241343,Bank


In [85]:
# determine venue counts
toronto_venues.groupby('Neighborhood').count() 

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,15,15,15,15,15,15
Berczy Park,3,3,3,3,3,3
Brockton,11,11,11,11,11,11
Central Bay Street,84,84,84,84,84,84
Christie,100,100,100,100,100,100
Design Exchange,8,8,8,8,8,8
Dovercourt Village,12,12,12,12,12,12
Dufferin,4,4,4,4,4,4
Exhibition Place,2,2,2,2,2,2


In [86]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))#manhattan_venues

There are 180 uniques categories.


#### Analyse neighborhoods

In [87]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# REVIEW:  move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()#categories

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Food,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Park,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [88]:
#REVIEW  fractions as Neighborhoods are joined - no Boroughs!
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Food,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Park,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.00000,0.000000,0.000000,0.066667,0.000000,0.066667,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.066667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [89]:
toronto_grouped.shape


(25, 180)

In [90]:

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3       Coffee Shop  0.07
4               Bar  0.07


----Berczy Park----
                             venue  freq
0                     Tennis Court  0.33
1                       Playground  0.33
2                       Restaurant  0.33
3  Molecular Gastronomy Restaurant  0.00
4                     Liquor Store  0.00


----Brockton----
                 venue  freq
0        Grocery Store  0.18
1  Fried Chicken Joint  0.09
2             Pharmacy  0.09
3          Pizza Place  0.09
4       Discount Store  0.09


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1  Italian Restaurant  0.05
2      Ice Cream Shop  0.05
3      Sandwich Place  0.04
4        Burger Joint  0.04


----Christie----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


In [91]:
#First, let's write a function to sort the venues in descending order - from lab.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

In [93]:
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [94]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

In [95]:
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [96]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boat or Ferry,Sculpture Garden,Bar,Boutique,Airport Food Court
1,Berczy Park,Restaurant,Playground,Tennis Court,Women's Store,Convenience Store,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
2,Brockton,Grocery Store,Beer Store,Pharmacy,Discount Store,Japanese Restaurant,Fast Food Restaurant,Pizza Place,Sandwich Place,Fried Chicken Joint,Coffee Shop
3,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Café,Burger Joint,Restaurant,Spa,Middle Eastern Restaurant,Bar
4,Christie,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Restaurant,Asian Restaurant,Breakfast Spot,Hotel,Gym


#### Create clusters

In [97]:
#Run *k*-means to cluster the neighborhood into 5 clusters. : why 5? previoyl 6 in Toronto: Metro Toronto – Etobicoke, Scarboroug,York, East York, North York, and 
#   the City of Toronto, into a new singular City of Toronto in 1998 but onl4 boroughs that have Toronto according to the csv

# set number of clusters
kclusters = 5#7#6#5#4#5 a number were tried but 5 seemed to yield the best results

In [98]:

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [99]:
krows = toronto_grouped_clustering
#print(krows)

In [100]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0,max_iter=300).fit(toronto_grouped_clustering)
#fit to try to get luster centres

In [101]:
#inspest result of k mens
#print(type(kmeans))
#print(kmeans.n_iter_))

In [102]:
print(kmeans.n_iter_)

2


In [103]:
# check cluster labels generated #for each row in the dataframe

kmeans.labels_[0:24]

array([0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 0,
       0, 0], dtype=int32)

In [106]:
# add clustering labels - only execute this once per session so you only add a single colun





#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [107]:
neighborhoods_venues_sorted.head()#neighborhood

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Adelaide,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boat or Ferry,Sculpture Garden,Bar,Boutique,Airport Food Court
1,0,Berczy Park,Restaurant,Playground,Tennis Court,Women's Store,Convenience Store,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
2,0,Brockton,Grocery Store,Beer Store,Pharmacy,Discount Store,Japanese Restaurant,Fast Food Restaurant,Pizza Place,Sandwich Place,Fried Chicken Joint,Coffee Shop
3,0,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Café,Burger Joint,Restaurant,Spa,Middle Eastern Restaurant,Bar
4,0,Christie,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Restaurant,Asian Restaurant,Breakfast Spot,Hotel,Gym


In [108]:
#df w/ neigghborhoods that end w/ Toronto
toronto_merged = df_am3 

In [110]:
toronto_merged.head()#verify

,Borough,Latitude,Longitude,Neighbourhood,PostalCode
0,Downtown Toronto,43.773136,-79.239476,Harbourfront,M1H
1,Downtown Toronto,43.744734,-79.239476,Regent Park,M1J
2,Downtown Toronto,43.803762,-79.363452,Ryerson,M2H
3,Downtown Toronto,43.778517,-79.346556,Garden District,M2J
4,Downtown Toronto,43.725882,-79.315572,St. James Town,M4A


In [111]:
toronto_merged.rename(columns={"Neighbourhood": "Neighborhood"}, inplace = True)

In [112]:
toronto_merged.head()#neighborhood

,Borough,Latitude,Longitude,Neighborhood,PostalCode
0,Downtown Toronto,43.773136,-79.239476,Harbourfront,M1H
1,Downtown Toronto,43.744734,-79.239476,Regent Park,M1J
2,Downtown Toronto,43.803762,-79.363452,Ryerson,M2H
3,Downtown Toronto,43.778517,-79.346556,Garden District,M2J
4,Downtown Toronto,43.725882,-79.315572,St. James Town,M4A


In [113]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood') 

In [114]:
toronto_merged.head() # check the last columns!

,Borough,Latitude,Longitude,Neighborhood,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.773136,-79.239476,Harbourfront,M1H,0,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Bank,Bakery,Women's Store,Dog Run,Discount Store
1,Downtown Toronto,43.744734,-79.239476,Regent Park,M1J,2,Playground,Women's Store,Cosmetics Shop,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Cuban Restaurant
2,Downtown Toronto,43.803762,-79.363452,Ryerson,M2H,0,Golf Course,Dog Run,Fast Food Restaurant,Pool,Mediterranean Restaurant,Athletics & Sports,Cuban Restaurant,Discount Store,Diner,Dessert Shop
3,Downtown Toronto,43.778517,-79.346556,Garden District,M2J,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Tea Room,Electronics Store,Japanese Restaurant,Bakery,Food Court,Toy / Game Store
4,Downtown Toronto,43.725882,-79.315572,St. James Town,M4A,0,Coffee Shop,Intersection,Financial or Legal Service,Hockey Arena,Portuguese Restaurant,Women's Store,Creperie,Discount Store,Diner,Dessert Shop


In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [116]:
# key error when run again!

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

#### Cluster 1

Food?

In [119]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,3] + list(range(5, toronto_merged.shape[1]))]]
#21 Neighborhoods

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,0,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Bank,Bakery,Women's Store,Dog Run,Discount Store
2,Downtown Toronto,Ryerson,0,Golf Course,Dog Run,Fast Food Restaurant,Pool,Mediterranean Restaurant,Athletics & Sports,Cuban Restaurant,Discount Store,Diner,Dessert Shop
3,Downtown Toronto,Garden District,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Tea Room,Electronics Store,Japanese Restaurant,Bakery,Food Court,Toy / Game Store
4,Downtown Toronto,St. James Town,0,Coffee Shop,Intersection,Financial or Legal Service,Hockey Arena,Portuguese Restaurant,Women's Store,Creperie,Discount Store,Diner,Dessert Shop
5,East Toronto,The Beaches,0,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Gym,Farmers Market,Indian Restaurant
6,Downtown Toronto,Berczy Park,0,Restaurant,Playground,Tennis Court,Women's Store,Convenience Store,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
7,Downtown Toronto,Central Bay Street,0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Café,Burger Joint,Restaurant,Spa,Middle Eastern Restaurant,Bar
8,Downtown Toronto,Christie,0,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Restaurant,Asian Restaurant,Breakfast Spot,Hotel,Gym
9,Downtown Toronto,Adelaide,0,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boat or Ferry,Sculpture Garden,Bar,Boutique,Airport Food Court
10,Downtown Toronto,King,0,Coffee Shop,Café,Restaurant,Beer Bar,Cocktail Bar,Seafood Restaurant,Italian Restaurant,Hotel,Fast Food Restaurant,Creperie


#### Cluster 2

Baseball?

In [122]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,3] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,The Danforth West,1,Baseball Field,Furniture / Home Store,Women's Store,Cosmetics Shop,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


#### Cluster 3

Playground?

In [123]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,3] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,Regent Park,2,Playground,Women's Store,Cosmetics Shop,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Cuban Restaurant


#### Cluster 4

Park?

In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0,3] + list(range(5, toronto_merged.shape[1]))]]
#1

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,East Toronto,Riverdale,3,Park,Convenience Store,Clothing Store,Cocktail Bar,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


#### Cluster 5

Drugstore?

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0,3] + list(range(5, toronto_merged.shape[1]))]]
#1

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,West Toronto,Exhibition Place,4,Drugstore,Rental Car Location,Women's Store,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Cuban Restaurant


#### Personal Opinions and Thoughts (Final step 3 of 3)

My instinct (assuming cost is not a major issue. Preferably, they should look at the Richmond neighborhood because of the number of venues nearby (nusing FourSquare). That said, the Toronto Central Bay Street, Union, and King neighbords should also be considered to expand options (as these have a "lot" of venues also close by according to FourSquare).  Moreover, all four of these neighborhoods are geographically proximate o each other (looking at the maps generated).   The majority of venues are food and drinks related so i adds to the convenience.  Furthermore since all belong to the Downtown Tornto borough it is not unreasonable to think there is access all kinds of ameneties (it is a safe bet that most offices are in the CBD) and (if it is anything like New York) there is no need to maintain a car (keeping costs down) if most things are walking distance or "easily" reachable by public transportation.  That said, the analyis would have to be done again and a different conclusion might be arrived at depending on the specified requirements.